In [1]:
import numpy as np
from astropy.modeling import models, fitting
import matplotlib.pyplot as plt
from pathlib import Path
from astropy.io import fits
#from astropy.wcs import WCS
import os
#import scipy.stats as stats
#import json
import pandas as pd
from fitsutil import *
from bc_imaging_analysis import imaging_analysis
from xraysky.scripts import events_imaging
from bcpsf import *
#from numba import jit
import pickle



/storage/home/ajo5182/.conda/envs/xhcd/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


This one is for windows

In [ ]:
eventPath = Path(
    r"Z:/Astro_BlackCAT\BlackCAT_Calibration_Data\LC_Calibration_Data\y2024-12-12\BC057_Ti_FF_233_HORI_0_0\Analysis\imaging_analysis_20241213\combined_events_th1_200.fits.gz"
)
d = fits.open(eventPath)

In [ ]:
eventPath = Path(
    r"/mnt/z/Astro_BlackCAT/BlackCAT_Calibration_Data/LC_Calibration_Data/y2024-12-12/BC057_Ti_FF_233_HORI_0_0/Analysis/imaging_analysis_20241213/combined_events_th1_200.fits.gz"
)
d = fits.open(eventPath)
cols = d[1].columns
(d[1].header)

In [ ]:
print(type(d))
evntlist_in_browser(eventPath)

In [ ]:
#signal, trash = splitEventList(d,3020,"sum")
signal, noiseShelf = splitEventList(d,550,"sum")
noiseShelf.columns = cols
noiseShelf.header = d[1].header
noiseShelf.writeto("/home/laurel/astro/testing/noiseshelf.fits.gz",overwrite=True)
signal.columns = cols
signal.header = d[1].header
signal.writeto("/home/laurel/astro/testing/signal.fits.gz",overwrite=True)

In [ ]:
noise = Spectrum("/home/laurel/astro/testing/noiseshelf.fits.gz",th1 = 200, th2=60)
noise.plotSpectrum(binsize=10)
sig = Spectrum("/home/laurel/astro/testing/signal.fits.gz",th1 = 200, th2=60)
sig.plotSpectrum(binsize=10)

In [ ]:
a,b = halfEventList(signal)

In [ ]:
random_a = random_sample_events_list(a,1500)
random_b = random_sample_events_list(b,1000)

In [ ]:
noise = np.array(noiseShelf.data)


In [ ]:
random_a.data = np.append(random_a.data,noiseShelf.data)
print(len(random_a.data))

In [ ]:
SampleHDU = fits.BinTableHDU.from_columns(cols)
random_a.columns = cols
random_a.header = d[1].header
random_a.writeto("/home/laurel/astro/testing/test.fits.gz",overwrite=True)


In [ ]:
test = fits.open("/home/laurel/astro/testing/test.fits.gz")
test[1].header

In [ ]:
test = Spectrum("/home/laurel/astro/testing/test.fits.gz",th1 = 200, th2=60)
test.plotSpectrum(binsize=10)
#out = test.filterEvents(sumrange=[0,550])
#out.plotSpectrum(binsize=10)


In [ ]:
imager= events_imaging.BC_Imaging()
imageOut = imager.evtlist2image(
    "/home/laurel/astro/testing/test.fits.gz","/home/laurel/astro/testing/test_reconstruct.fits.gz"
)
imager.plotSource(imageOut)
peaks = imager.imager.findpeaks(imageOut.data.T) # Index 5 has the significance

In [ ]:
d1 = fits.open("/scratch/ajo5182/psfSamples/image_reconstruction/BC057_Ti_FF_233_HORI_15_0_samp1.fits.gz")
d2 = fits.open("/scratch/ajo5182/psfSamples/image_reconstruction/BC057_Ti_FF_233_HORI_15_0_samp2.fits.gz")


In [ ]:
test,cov1 = analyze_localization(d1,plot=True)

In [ ]:
test2,cov2 = analyze_localization(d2,plot=True)

In [ ]:
test2 = analyze_localization_directory("/home/laurel/astro/testing/test_reconstruct.fits.gz")

In [ ]:
#test = pd.DataFrame.from_dict(test,orient= 'index')
test2 = pd.DataFrame.from_dict(test2,orient= 'index')

In [ ]:
a = [None]*2
a[0] =test
a[1] = test2

In [ ]:
log = np.empty([len(folder_list),2]) # create empty array to hold a log of stuff


for windows

In [ ]:
# for windows
fileDirectory = r"Z:\Astro_BlackCAT\BlackCAT_Calibration_Data\LC_Calibration_Data\y2024-12-12"

for running on WSL


In [ ]:
fileDirectory = r"/mnt/z/Astro_BlackCAT/BlackCAT_Calibration_Data/LC_Calibration_Data/y2024-12-12"
outDirectory = r"/mnt/z/Astro_BlackCAT/BlackCAT_Calibration_Data/LC_Calibration_Data/psfSamples"
splitThreshold = 550
numberOfRandomSamples = 1500
folder_list = [folder for folder in os.listdir(fileDirectory) if os.path.isdir(os.path.join(fileDirectory, folder))]


For Roar

In [ ]:
fileDirectory = r"/storage/group/adf15/default/Astro_BlackCAT/BlackCAT_Calibration_Data/LC_Calibration_Data/y2024-12-12"
outDirectory = r"/scratch/ajo5182/psfSamples" 
splitThreshold = 550
numberOfRandomSamples = 1500
folder_list = [folder for folder in os.listdir(fileDirectory) if os.path.isdir(os.path.join(fileDirectory, folder))]

In [ ]:
#folder_list = [folder for folder in os.listdir(fileDirectory) if os.path.isdir(os.path.join(fileDirectory, folder))]

log = [[None for _ in range(2)] for _ in range(len(folder_list))] #np.empty([len(folder_list),2],str) # empty array to log errors and wahtnot

for a in range(len(folder_list)):
    eventlistPath = Path(fileDirectory)/Path(folder_list[a])/Path("Analysis/imaging_analysis_20241213/combined_events_th1_200.fits.gz")
    try:
        d = fits.open(eventlistPath)
        cols = d[1].columns
        header = d[1].header
    
        signal, noiseShelf = splitEventList(d,splitThreshold,"sum") # separate the noise shelf from the "signal data"
        signal_1,signal_2 = halfEventList(signal) # split the portion of the data that is the "signal" in half

        signal_1_random = random_sample_events_list(signal_1,numberOfRandomSamples) # take a random sample of each half of the split data
        signal_2_random = random_sample_events_list(signal_2,numberOfRandomSamples) # take a random sample of each half of the split data
    
        noise = np.array(noiseShelf.data) # make a numpy array from the noise shelf data

        signal_1_random.data = np.append(signal_1_random.data,noiseShelf.data) # add the noise shelf to each half of the data
        signal_2_random.data = np.append(signal_2_random.data,noiseShelf.data) # add the noise shelf to each half of the data

        signal_1_random.columns = cols
        signal_1_random.header = header

        signal_2_random.columns = cols
        signal_2_random.header = header

        signal_1_random.writeto(Path(outDirectory)/Path(folder_list[a]+"_samp1"+".fits.gz"),overwrite=True)
        signal_2_random.writeto(Path(outDirectory)/Path(folder_list[a]+"_samp2"+".fits.gz"),overwrite=True)

        #log[a,1] = folder_list[a]
        #log[a,2] = "created random sample files"
    except:
        #log[a,0] = folder_list[a]
        #log[a,1] = " no file or other error"
        print(folder_list[a]+" bad")


For WSL

In [2]:
data_directory = fileDirectory = r"/storage/group/adf15/default/Astro_BlackCAT/BlackCAT_Calibration_Data/LC_Calibration_Data/y2024-12-12"

image_directory= r"/mnt/z/Astro_BlackCAT/BlackCAT_Calibration_Data/LC_Calibration_Data/psfSamples/image_reconstruction"
file_list = [file for file in os.listdir(data_directory) if os.path.isfile(os.path.join(data_directory, file))]



In [ ]:
range(38,208)

For Roar


In [2]:
data_directory = r"/scratch/ajo5182/psfSamples" 

image_directory= r"/scratch/ajo5182/psfSamples/image_reconstruction"
file_list = [file for file in os.listdir(data_directory) if os.path.isfile(os.path.join(data_directory, file))]

In [ ]:
imager= events_imaging.BC_Imaging()

 
for a in range(len(file_list)): #range(len(file_list[38:208]))
    eventlist = Path(data_directory)/Path(file_list[a])
    image = Path(image_directory)/Path(file_list[a])
    imageOut = imager.evtlist2image(
        eventlist, image
    )

In [3]:
reconstruction_list = sorted([file for file in os.listdir(image_directory) if os.path.isfile(os.path.join(image_directory, file))])
results_list = [None]*len(reconstruction_list)


In [4]:
for a in range(len(reconstruction_list)):
    eventlistPath = Path(image_directory)/Path(reconstruction_list[a])
    d = fits.open(eventlistPath)
    print(a)
    results_list[a] = analyze_localization(d)



0
issue with finding peaks
1
issue with finding peaks
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
issue with finding peaks
33
issue with finding peaks
34
issue with finding peaks
35
issue with finding peaks
36
37
38
39
40
41
42
43
44
45
46
issue with finding peaks
47
issue with finding peaks
48
49
50
51
52
53
54
issue with finding peaks
55
issue with finding peaks
56
57
58
59
60
61
62
issue with finding peaks
63
issue with finding peaks
64
65
66
67
68
69
70
issue with finding peaks
71
issue with finding peaks
72
73
74
75
76
77
78
issue with finding peaks
79
issue with finding peaks
80
81
82
83
84
85
86
issue with finding peaks
87
issue with finding peaks
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
issue with finding peaks
131
issue with finding peaks
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
15

In [7]:
pickle.dump(results_list, "gaussfit_9-22.pkl")

TypeError: file must have a 'write' attribute

In [10]:
df = pd.DataFrame(results_list)
df.to_pickle("gaussfit_9-22.pkl")

In [9]:
result = pickle.load("gaussfit_9-22.pkl")

TypeError: file must have 'read' and 'readline' attributes

In [6]:
v = sorted(reconstruction_list)